In [ ]:
!pip install beautifulsoup4
!pip install requests

In [5]:
# 마운트 -> 구글 드라이브를 불러 작업할 디렉토리(현재 파일의 위치)를 지정해준다
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Crawling
!pwd
!ls

Mounted at /content/drive
/content/drive/MyDrive/Crawling
/content/drive/MyDrive/Crawling
BeautifulSoup.ipynb


In [21]:
# 기본 설명용 코드
import requests
from bs4 import BeautifulSoup

url = 'https://github.com/search?p=1&q=crawling&type=Repositories'

# 웹페이지에 데이터 요청을 보냄
response = requests.get(url)

# html코드중 올바른 반응이왔을때 (404에러같은게 뜨거나 서버에서 블록한게 아닌)
if response.status_code == 200:
    # html 코드 그대로 받음
    html = response.text
    # xml이나 json으로 구성된 페이지가 아니므로 html파서로 beautifulsoup라이브러리용 데이터 변환
    soup = BeautifulSoup(html, 'html.parser')

    # 가장 위의 태그 하나만 받는 코드들
    test=soup.h3
    print(test)
    test=soup.find('h3')
    print(test)

    # html태그제외한 텍스트만 추출하는 방법
    test=soup.find('h3').get_text()
    print(test)
    test=soup.find('h3').string
    print(test)

    # 조건에 맞는 모든 태그들 받는 코드들
    test=soup.find_all('h3')
    print(test)
    attrs = {'class': 'v-align-middle'}
    test = soup.find_all('a', attrs=attrs)
    print(test)
    # all로 받은 태그 리스트에서 for문으로 텍스트만 추출하려면
    for code in test:
        print(code.get_text())

else : 
    print('연결되지않음')
    print(response.status_code)

<h3 class="Box-title">Search cheat sheet</h3>
<h3 class="Box-title">Search cheat sheet</h3>
Search cheat sheet
Search cheat sheet
[<h3 class="Box-title">Search cheat sheet</h3>, <h3>
    25,043 repository results


</h3>]
[<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":1,"per_page":10,"query":"crawling","result_position":1,"click_id":529502,"result":{"id":529502,"global_relay_id":"MDEwOlJlcG9zaXRvcnk1Mjk1MDI=","model_name":"Repository","url":"https://github.com/scrapy/scrapy"},"originating_url":"https://github.com/search?p=1&amp;q=crawling&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="1b5491ef94832c63cf7f48882b5fbb9ab7a2f0c7ab35aed2d82b7c56b54cbc3f" href="/scrapy/scrapy">scrapy/scrapy</a>, <a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":1,"per_page":10,"query":"crawling","result_position":2,"click_id":54357610,"result":{"id":54357610,"global_relay_id":"MD

In [16]:
import requests
from bs4 import BeautifulSoup
import csv
import time

users = []
repositories = []
descriptions = []
stars = []
languages = []

# csv 제목 지정
f = open("data.csv", "w", encoding='utf-8')
writer = csv.writer(f)
data = [['user', 'repository', 'descripsion', 'star', 'language']]
writer.writerows(data)


# 데이터를 위 users, repositories, descriptions, stars, languages 리스트에 저장하는함수
def get_data(url):
  
    # url = 'https://github.com/search?p=1&q=crawling&type=Repositories'
    response = requests.get(url)
    title_all = []

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        # 제목의 class를 이용해 제목받아옴.
        attrs = {'class': 'v-align-middle'}
        title = soup.find_all('a', attrs=attrs)
        # print(title)
        for i in title:
            # 제목은 scrapy/scrapy 형식이고, 나중에 이를 쓸일이 있기에 title_all에 저장, 그리고 '/'로 스플릿하여 유저명과 리포지토리명 리스트에 저장
            title_all.append(i.get_text())
            titles = i.get_text().split('/')
            users.append(titles[0])
            repositories.append(titles[1])
        print(users)
        print(repositories)


        # 설명부분의 class를 이용해 설명 받아옴. 텍스트를 뽑으면 \n과 공백들이 많아서 strip()으로 앞뒤 불필요요소 제거함.(처음엔 공백과 \n의 개수가 일정해서 인덱스로 가져왔었음.)
        attrs = {'class': 'mb-1'}
        description = soup.find_all('p', attrs=attrs)
        # print(description)
        for i in description:
            # descriptions.append(i.get_text()[9:-7])
            descriptions.append(i.get_text().strip())
        print(descriptions)


        # 별 갯수 가져오는데 class가 Link--muted인걸 가져와보니 class가 Link--muted f6인 '30 issues need help' 같은 문자도 같이 옴
        # 그래서 class와 href 둘다 만족하는 조건으로 star가져옴. href가 /scrapy/scrapy/stargazers 로 '/' + 제목전체값 + '/stargazers'이기에 아까 title_all을 따로 저장했다 가져옴
        # print(title_all)
        for i in range(len(title_all)):
            href = '/' + title_all[i] + '/stargazers'
            # print(href)
            attrs = {'class': 'Link--muted', 'href': href}
            star = soup.find('a', attrs=attrs)
            # stars.append(star.get_text()[18:-13])
            stars.append(star.get_text().strip())
        print(stars)

        # 프로그래밍언어는 itemprop 값이 programmingLanguage인걸로 찾아옴.
        attrs = {'itemprop': 'programmingLanguage'}
        language = soup.find_all('span', attrs=attrs)
        # print(language)
        for i in language:
            languages.append(i.get_text())
        print(languages)

    else:
        print('연결되지않음')
        print(response.status_code)


# 키워드입력
keyword = input()

# 5페이지까지 키워드를 넣어서 검색하여 users, repositories, descriptions, stars, languages 리스트에 저장
for i in range(1, 6):
    url = 'https://github.com/search?p='+str(i)+'&q='+keyword+'&type=Repositories'
    # 위에 정의한 리스트에 요소들 추가하는 함수(1~5페이지까지 돌면서 append함)
    get_data(url)
    # 빠른접근시 문제되어 10초간 여유 둠. 최적시간을 알면 줄일 수 있을듯.
    time.sleep(10)

# csv에 위 설정한 제목에 맞게끔 줄별로 입력
for i in range(len(repositories)):
    row = [[users[i], repositories[i], descriptions[i], stars[i], languages[i]]]
    writer.writerows(row)

# 다 입력하고 파일 닫음
f.close()

crawling
['scrapy', 'dotnetcore', 'waditu', 'qinxuye', 'gaojiuli', 'crawljax', 'qiyaTech', 'mjhea0', 'crawl', 'chensoul']
['scrapy', 'DotnetSpider', 'tushare', 'cola', 'gain', 'crawljax', 'javaCrawling', 'Scrapy-Samples', 'crawl', 'scrapy-zhihu-github']
['Scrapy, a fast high-level web crawling & scraping framework for Python.', 'DotnetSpider, a .NET standard web crawling library. It is lightweight, efficient and fast high-level web crawling & s…', 'TuShare is a utility for crawling historical data of China stocks', 'A high-level distributed crawling framework.', 'Web crawling framework based on asyncio.', 'Crawljax: Crawling Dynamic (JavaScript-based) Web Applications', '"奇伢爬虫"是基于sprint boot 、 WebMagic 实现 微信公众号文章、新闻、csdn、info等网站文章爬取，可以动态设置文章爬取规则、清洗规则，基本实现了爬取大部分网站的文章。', 'Scrapy examples crawling Craigslist', 'Dungeon Crawl: Stone Soup official repository', 'scrapy examples for crawling zhihu and github']
['43.4k', '3.3k', '11.5k', '1.4k', '2k', '459', '294', '192', '1.7k', '217']
['Pyth